In [1]:
import ctypes 
import subprocess
import numpy as np


In [2]:
env_path = 'r:/Projects/trunk/LockOnExe/bin/x86_64/vc143.debug-mt/dcs.exe'
path = 'r:/Projects/trunk/LockOnExe/bin/x86_64/vc143.debug-mt/RLConnector.dll'

In [3]:
subprocess.Popen([env_path, "-dofile", "tank.miz"])

<Popen: returncode: None args: ['r:/Projects/trunk/LockOnExe/bin/x86_64/vc14...>

In [4]:
# Загрузка библиотеки
RLConnector = ctypes.CDLL(path)


In [5]:
class C_RLState(ctypes.Structure):
    _fields_ = [('pos_x', ctypes.c_double),
                    ('pos_y', ctypes.c_double),
                    ('pos_z', ctypes.c_double),
                    ('v_x', ctypes.c_double),
                    ('v_y', ctypes.c_double),
                    ('v_z', ctypes.c_double),
                    ('course', ctypes.c_double)]
    
    def to_numpy(self):
        return np.array([self.pos_x, self.pos_z, self.v_x, self.v_z, self.course, 0])
          
class C_RLAction(ctypes.Structure):
    _fields_ = [
        ('speed', ctypes.c_float),
        ('dir_move', ctypes.c_float),
        ('dir_view', ctypes.c_float),
        ('is_move_to_view', ctypes.c_int),
        ('state', ctypes.c_int),
        ('action', ctypes.c_int),
    ]

    def from_numpy(self, array):
        self.is_move_to_view = 1 #int(array[0])
        self.dir_move = float(array[1])
        self.dir_view = float(array[2])
        self.speed = float(array[3])
        self.state = 0 #int(array[4])
        self.action = 0 #int(array[4])

    def to_numpy(self):
        return np.array([self.is_move_to_view, self.dir_move, self.dir_view, self.speed, self.state, self.action])

In [6]:
RLConnector.reward.restype = ctypes.c_float

RLConnector.agents_count.restype = ctypes.c_int

RLConnector.agent_id.restype = ctypes.c_int
RLConnector.agent_id.argtypes = [ctypes.c_int,]

RLConnector.current_state.restype = C_RLState

RLConnector.is_ready_to_start.restype = ctypes.c_bool

RLConnector.set_action.argtypes = [ctypes.c_int, C_RLAction,]

In [14]:
RLConnector.is_ready_to_start()

True

In [8]:
action_to = C_RLAction()
action_to.from_numpy([1., -0.9, -0.9, 1, 0., 0.])
action_to.to_numpy()

array([ 1.        , -0.89999998, -0.89999998,  1.        ,  0.        ,
        0.        ])

In [11]:
RLConnector.set_action(0, action_to)

-1900806136

In [7]:
res = RLConnector.current_state()

In [8]:
res.to_numpy()

array([0., 0., 0., 0., 0., 0.])

In [9]:
RLConnector.agents_count()

0

In [10]:
RLConnector.agent_id(0)

-1

In [11]:
RLConnector.set_pause()

0

In [10]:
RLConnector.set_start()

0

In [12]:
RLConnector.set_restart()

0

In [5]:
del RLConnector

NameError: name 'RLConnector' is not defined

In [7]:
RLConnector._handle

140735128535040

In [12]:
dll_handle = ctypes.windll.kernel32.LoadLibraryW(path)
dll_handle

1935147008

In [13]:
ctypes.windll.kernel32.FreeLibrary(dll_handle)

0

In [3]:
def is_dll_loaded(dll_name):
    try:
        ctypes.WinDLL(dll_name)
        return True
    except OSError:
        return False


dll_loaded = is_dll_loaded(path)
dll_loaded

True